In [ ]:
!pip install jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import jieba
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
base_dir = "/content/drive/MyDrive/naive_b_news/chinese_news_trans"
test_dir = "/content/drive/MyDrive/naive_b_news/chinese_news_test"

In [ ]:
def process_dirs(base_dir):
    df = pd.DataFrame(columns = ["類別", "內容"])
    # os.walk 會走到沒檔案才停下
    for dir_path, dir_names, file_names in os.walk(base_dir):
        for single_file in file_names:
            if not single_file.startswith("."):
                f = open(os.path.join(dir_path, single_file), "r", encoding = "utf-8")
                content = f.read()
                # 讀完檔後做出第一步處理,先把"|\r", "\n"都去掉
                content = content.replace("\r", "").replace("\n", "")
                split_word = jieba.cut(content)
                #分詞
                content = " ".join(split_word)
                s = pd.Series([dir_path.split("/")[-1], content], index = ["類別", "內容"])
                df = df.append(s, ignore_index = True)
    df["類別"] = df["類別"].astype("category")
    return df       

In [ ]:
df = process_dirs(base_dir)
df

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 1.072 seconds.
DEBUG:jieba:Loading model cost 1.072 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


,類別,內容
0,教育,特寫 ： 奉獻者 心聲 引起 共鳴 新華社 杭州...
1,教育,無錫市 形成 青少年 德育教育 網絡 新華社 南...
2,教育,《 少年 知識 大全 》 等 書 贏得 好 評新...
3,教育,【 日 期 】 19960526 【 版 號 】 4 【 ...
4,教育,無錫市 形成 青少年 德育教育 網絡 新華社 南...
...,...,...
2632,環境,政府 部門通 過電 臺 首次 進行 專題 調查 ...
2633,環境,孟計劃 保護 珍稀 老虎 新華 社達卡 ５ 月 ...
2634,環境,美對 破壞 生態 環境者 處以 重罰 新華 社華...
2635,環境,我國 ２ ６ 個省 和 自治 區開展 省級 國土...


In [ ]:
"""
由於 scikit-learn 不接受字串，所以我們一定要把類別轉換成整數
你可以使用 cat.categories 得到所有類別
再使用 cat.codes 轉換成整數

"""
saved_map = {cat:df["類別"].cat.categories.get_loc(cat)
        for cat in df["類別"].cat.categories}
saved_map

{'交通': 0,
 '政治': 1,
 '教育': 2,
 '環境': 3,
 '經濟': 4,
 '藝術': 5,
 '計算機': 6,
 '軍事': 7,
 '醫藥': 8,
 '體育': 9}

In [ ]:
# 把我們的測試資料讀取，並且使用剛剛存起來的 category 來 map
df["類別"] = df["類別"].cat.codes
df

,類別,內容
0,2,特寫 ： 奉獻者 心聲 引起 共鳴 新華社 杭州...
1,2,無錫市 形成 青少年 德育教育 網絡 新華社 南...
2,2,《 少年 知識 大全 》 等 書 贏得 好 評新...
3,2,【 日 期 】 19960526 【 版 號 】 4 【 ...
4,2,無錫市 形成 青少年 德育教育 網絡 新華社 南...
...,...,...
2632,3,政府 部門通 過電 臺 首次 進行 專題 調查 ...
2633,3,孟計劃 保護 珍稀 老虎 新華 社達卡 ５ 月 ...
2634,3,美對 破壞 生態 環境者 處以 重罰 新華 社華...
2635,3,我國 ２ ６ 個省 和 自治 區開展 省級 國土...


In [ ]:
test_df = process_dirs(test_dir)
test_df

,類別,內容
0,藝術,藝術橋 樑 溝通 海外 鄉誼 精品 迭出 活躍群眾 生活 台山 文化 打僑牌 聚僑心 ...
1,藝術,文化 事業 健康 繁榮 “ 九五 ” 期間 ， 我市 文化 領域 堅持 ...
2,藝術,劉長 安 ： 作曲家 ， 文化 行政 管理 工作者 。 山東省 黃 縣 人 。 19...
3,藝術,宣武 區 “ 十五 ” 期間 文化 事業 發展 計劃 一 、 “ 九五 ” 時期 文化 ...
4,藝術,關益 全 ： 藝苑 拾影 三十年 姓名 ： 關益 全 ， 年齡 ： 59 歲 ， 職務 ...
...,...,...
96,交通,日 月光 華 -- Traffic _ Info 精華區 文章 閱讀 -------...
97,交通,日 月光 華 -- Traffic _ Info 精華區 文章 閱讀 -------...
98,交通,日 月光 華 -- Traffic _ Info 精華區 文章 閱讀 -------...
99,交通,三趟 火車 停開 乘客 可 全額 退票 瀏覽次數 ： 1180 ...


In [ ]:
# 替換測試類別
# 這邊必須使用剛剛存起來的字典來替換
# 因為如果直接使用 code 可能會發生沒對照到的事故
test_df["類別"] = test_df['類別'].replace(saved_map)
test_df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
# 一定要使用 fit_transform才會幫你轉換成詞向量
bag = vec.fit_transform(df["內容"])
print("總共維度:", len(vec.get_feature_names_out()))
vec.get_feature_names_out()

總共維度: 96043


array(['00', '000', '00000001', ..., '龔谷青', '龜板', '龜茲古韻'], dtype=object)

In [ ]:
# 只要是文字, 我們通常就會選擇 MultinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.001).fit(bag, df["類別"])
clf

MultinomialNB(alpha=0.001)

In [ ]:
# 由於我們用剛剛的vec訓練他,所以維度會保持跟剛剛一樣
test_bag = vec.transform(test_df['內容'])
print("維度:", len(vec.get_feature_names_out()))
vec.get_feature_names_out()

維度: 96043


array(['00', '000', '00000001', ..., '龔谷青', '龜板', '龜茲古韻'], dtype=object)

In [ ]:
from sklearn.metrics import accuracy_score
predict = clf.predict(test_bag)
print("預測:", list(predict))
print("正確標籤:", list(test_df['類別']))
print("Naive-Bayes 正確率: ", accuracy_score(test_df['類別'], predict) * 100, "%")

預測: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
正確標籤: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Naive-Bayes 正確率:  100.0 %


In [ ]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=8)
clf = clf.fit(bag, df['類別'])
predict = clf.predict(test_bag)
print("預測:", list(predict))
print("正確標籤:", list(test_df['類別']))
print("kNN 正確率: ", accuracy_score(test_df['類別'], predict) * 100, "%")


預測: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
正確標籤: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
kNN 正確率:  100.0 %


由於訓練資料非常完善，而且目標相對單純，所以你發現不管單純貝氏或者 kNN 都可以順利達到高正確率